In [4]:
# %%
from openai import OpenAI
from dotenv import load_dotenv
import json

client = OpenAI()
# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")



True

In [5]:
# %%
def build_prompt(context: str) -> str:
    prompt_template = """You are a proffesional podcast assistant creating a test for advanced listeners. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}""".strip()

    prompt = prompt_template.format(context=context).strip()
    return prompt


def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


def rag(context: str):
    prompt = build_prompt(context)
    answer = llm(prompt)
    return json.loads(answer)


def add_qa_to_transcripts(transcripts):
    """Add a "question" and "answer" prepared by llm to each transcript in the list of transcripts"""
    new_transcripts = []
    for transcript in transcripts:
        new_qa = rag(transcript["text"])
        new_transcript = {**transcript, **new_qa}
        new_transcripts.append(new_transcript)
    return new_transcripts



In [6]:
# %%


with open("../data/external/transcript_qna.json", "r") as f:
    data = json.load(f)


def update_answer(data):
    for d in data:
        if d["answer"] == "NA":
            context = d["text"]
            new_answer = rag(context)
            d["answer"] = new_answer
    return data



In [ ]:
# %%
test_transcript = test_transcripts[0]["text"]

test_rag = rag(test_transcript)

In [ ]:
# %%
transcripts_with_qa = add_qa_to_transcripts(transcripts)
for t in transcripts_with_qa:
    print(json.dumps(t, indent=4), end="\n\n")


In [ ]:
# %%

with open("../data/external/transcript_qna.json", "r") as f:
    to_replace = json.load(f)
